In [1]:
! pip install natasha
! pip install pyTelegramBotAPI
! pip install requests
! pip install stanza
! pip install gtts
! pip install pywttr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from natasha import (Segmenter,
                     NewsEmbedding,
                     NewsMorphTagger,
                     NewsSyntaxParser,
                     NewsNERTagger,
                     Doc,
                     MorphVocab)

segmenter = Segmenter()

emb = NewsEmbedding()

morph_tagger = NewsMorphTagger(emb)

syntax_parser = NewsSyntaxParser(emb)
morph_vocab = MorphVocab()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#Импорт Stanza
import stanza
# Скачивание требуемой модели
stanza.download('ru') 
# инициализация модели
nlp = stanza.Pipeline('ru') 


INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [5]:
from pywttr import Wttr

In [6]:
from gtts import gTTS
import os
import asyncio
import json
import telebot
import time
from telebot import types
import re
from geopy import geocoders
import requests as req
import pandas as pd
from natasha import LOC
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import datetime
from natasha import DatesExtractor
dates_extractor = DatesExtractor(morph_vocab)
morph_vocab = MorphVocab()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


### Оттачивание опеределения функции посредством составления тематических словарей фраз для прогона через tfidf 

In [24]:
#апи потом перенести в отдельный файл, чтобы ни у кого кроме нас не было доступа
token = '6051155396:AAHArds26m8s9-hjRfWpFhISkiYURPD5Tak'


In [10]:
import pymorphy2
from nltk.corpus import stopwords # может и не понадобятся
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=str.split)
import numpy as np
from scipy.spatial.distance import cdist

from nltk.tokenize import RegexpTokenizer, sent_tokenize
tokenizer = RegexpTokenizer(r'[А-ЯЁа-яё]+')
morph = pymorphy2.MorphAnalyzer()

#собираем свой микрокорпус для определения интенции и каждому "классу" присваиваем метку класса

say_hi = {'text': ['привет',
             'здаров',
             'здароф',
             'здравствуй',
             'хай',
             'добрый день',
             'приветствую',
             'шалом',
             'добрый вечер',
             'доброе утро',
             'мир вам',
             'тебе привет',
             'рота, подъём ',
             'прив',
             'хеллоу',
             'день добрый',
             'вечер добрый'
             ],
             'value': [0 for i in range(17)]}

weather = {'text': ['что там на улице',
                        'как там погода',
                        'сколько сейчас градусов',
                        'что там за окном',
                        'какая погода сейчас в Москве',
                        'чего мне ждать завтра',
                        'подскажи погоду на завтра',
                        'что по погоде',
                        'смогу ли я завтра погулять',
                        'ясно или не очень',
                        'как там погодка',
                        'давно не выходила на улицу, знать бы что там за окном',
                        'будет ли завтра дождик',
                        'сейчас бы пойти погулять',
                        'хочу узнать погоду в спб',
                        'хочу выйти погулять',
                        'как насчёт путешествия в другой город'
                        ],
            'value': [1 for i in range(17)]}
voice = {'text': ['переведи мне в текст фразу',
             '',
             'следи за часами',
             'озвучь пожалуйста',
             'держи сообщение',
             'хочу услышать фразу',
             'переведи в звук',
             'скажи голосом',
             'я соскучился по человеческому голосу',
             'произнеси фразу',
             'озвучь предложение',
             'лень читать, хочу послушать эту фразу',
             'давно я не слышал человеческой речи',
             'не хочу читать, переведи текст в аудио',
             'войс',
             'аудио',
             'озвучка',
             'соскучился по голосу',
             'подать голос',
             'переведи текст в аудио'
             ],
             'value': [2 for i in range(20)]}
say_bye = {'text': ['пока',
             'прощай',
             'до скорой встречи',
             'спокойной ночи',
             'до свидания',
             'досвидос',
             'покеда',
             'отбой',
             'выключайся',
             'до встречи',
             'пакеда',
             'бай',
             'гудбай',
             'увидимся',
             'бывай'
             ],
             'value': [3 for i in range(15)]}

def preprocess(text):
    tokens = tokenizer.tokenize(text.lower())  # и токенизируем
    lem = []  # для каждого документа создаём список и пополняем его леммами
    for word in tokens:
        lem.append(morph.parse(word)[0].normal_form)
    return ' '.join(lem) # соединяем леммы через пробел

#создаём датасет для каждого паттерна, что сохранить их как для классификации
say_hi_df = pd.DataFrame(data=say_hi)
say_bye_df = pd.DataFrame(data=say_bye)
weather_df = pd.DataFrame(data=weather)
prep_weather_df = weather_df.copy()
#кроме того, лемматизируем для основных наших функций, чтобы улучшить идентификацию запрашиваемой функции
#так как всё равно мы будем искать по лемме

'''
для возможного улучшения поиска
лемматизируем наши тематические функции, отличные от приветствия и прощания
'''
prep_weather_df['text'] = prep_weather_df['text'].apply(lambda text: preprocess(text))

voice_df = pd.DataFrame(data=voice)
prep_voice_df = voice_df.copy()
prep_voice_df['text'] = voice_df['text'].apply(lambda text: preprocess(text))

frames = [say_hi_df, weather_df, prep_weather_df, voice_df, prep_voice_df, say_bye_df]
patterns_df = pd.concat(frames, axis=0)
patterns_df = patterns_df.drop_duplicates(subset=['text'])


def vectorization(corpus):
    '''
    векторизуем наш микрокорпус для определения интенции
    '''
    X = vectorizer.fit_transform(corpus)
    return X.toarray()

def query_vector(query): 
    '''
    аналогично векторизуем запрос
    '''
    query = preprocess(query)
    vect = vectorizer.transform([query])
    return vect.toarray()

def dist(corpus, matr_tfidf, v_query):
    '''
    умножаем вектор запроса на вектор корпуса,
    чтобы узнать косинусную близость запроса к вектору каждой строчки
    '''
    corpus['res'] = cdist(matr_tfidf, v_query, metric='cosine') 
    A = np.array(corpus['res']) # переводим в numoy array так как с ним удобнее работать
    ind = np.argsort(A, axis=0) # сортируем по наиболей кос близости
    B = np.array(corpus['value'])
    return np.take_along_axis(B, ind, axis=0).tolist() # по отсортированным косинусным штукам сортируем всю остальную таблицу с текстом 

def identify_query(query):
    matr_tfidf = vectorization(patterns_df['text'])
    v_query = query_vector(query)
    distance = dist(patterns_df, matr_tfidf, v_query) 
    return distance[0] # в конце выдаём первое значение, так у него после сортировки должна быть наибольшая релевантность нашему сообщению

In [11]:
#работа с сокращениями
cuts = {'Санкт-Петербург':['спб', 'питер'],
 'Москва':['мск'],
 'Нижний Новгород':['нино', 'нн','нижнов', 'нижний'],
 'Екатеринбург':['екб'],
 'Новосибирск':['нск']}
for en, val in cuts.items():
  print(en, val)

Санкт-Петербург ['спб', 'питер']
Москва ['мск']
Нижний Новгород ['нино', 'нн', 'нижнов', 'нижний']
Екатеринбург ['екб']
Новосибирск ['нск']


In [12]:
times_marks = {'today':['сейчас','сегодня'],
               'tomorrow':['завтра'],
               'afrer_tomorrow':['послезавтра']
               }

In [22]:
bot = telebot.TeleBot(token)
@bot.message_handler(commands=['start'])
def start_message(message):
  
  sent = bot.send_message(message.chat.id, 'можешь попробовать поздороваться или сразу выкладывай по делу')
  bot.register_next_step_handler(sent, getting_text)


def getting_text(message):
    
    global modus
    '''
    на практике получается так, что бот видит латиницу наиболее близкой к приветствию
    кроме того, вроде как подключение этой функции требует некоторой асинхронности, с которой я не разобралась
    поэтому этот эксперимент закомменчен
    '''
    # if not is_russian(message.text): # ещё цифры должен обрабатывать
    #     letter = bot.send_message(message.chat.id, 'ты что-то не то ввёл, возможно на английском')
    #     bot.register_next_step_handler(letter, getting_text)
    modus = identify_query(message.text)
    
    if modus == 0:
        # приветствие
        bot.send_message(message.chat.id, 'напоминаю. я могу подсказать погоду или озвучить твоё сообщение')
        sent = bot.send_message(message.chat.id, 'чего ты больше хочешь?')
        bot.register_next_step_handler(sent, getting_text)
    
    elif modus == 1:
      city = []
      city.append(getting_loc(message.text))
      if len(city) == 0 or city is None or city == [None]:
        # вынужденный костыль, так как natasha не видит метку LOC вне контекста или когда название города написано с маленькой буквы
        city_req = bot.send_message(message.chat.id,'повтори, пожалуйста, запрос, но с названием города с большой буквы')
        bot.register_next_step_handler(city_req, getting_text)
        pass
      else:
        #пока просто узнаём прогноз для города
        try:
          wttr = Wttr(city[0])
          forecast = wttr.ru()
        except Exception as e: 
          # отработка сценария, если это не существующий город
          bot.send_message(message.chat.id,'ты уверен, что это существующий город?')    
          request_again = bot.send_message(message.chat.id, 'хочешь чего-то ещё?')
          bot.register_next_step_handler(request_again, getting_text)    
          

      # берём дату
      date_point = []
      if getting_timing(message.text) == 'today':
        day = 'today'
        ind = 0
      elif getting_timing(message.text) == 'tomorrow':
        day = 'tomorrow'
        ind = 1
      elif getting_timing(message.text) == 'afrer_tomorrow':
        day = 'afrer_tomorrow'
        ind = 2
      else:
        date_point = getting_timing(message.text) # возвращает список от которого мы впоследствии возьмём точную метку времени
        if len(date_point) == 0 or date_point == [None]:
          day = 'today'
          ind = 0
        else: 
          today = datetime.date.today()
          ind = abs((today-date_point[0]).days)
          if ind == 0:
             day = 'today'
          elif ind == 1: 
            day = 'tomorrow'
          elif ind == 2: 
            day = 'afrer_tomorrow'
          elif ind >= 3:
              # отработка далёкой даты, когда мы не можем запросить по нашему апи погоды 
              bot.send_message(message.chat.id,'так далеко смотреть я ещё не умею')    
              request_again = bot.send_message(message.chat.id, 'хочешь чего-то ещё?')
      # один сценарий отработки, так как  апи не работает с более далёкими датами, но зато попутно определяет это сегодня завтра или послезавтра 
      bot.send_message(message.from_user.id, f'в городе {city[0]} {times_marks[day][0]} на улице температура {forecast.weather[ind].avgtemp_c}')
      request_again = bot.send_message(message.chat.id, 'хочешь чего-то ещё?')
      bot.register_next_step_handler(request_again, getting_text)   

    
    elif modus == 2:
      for_sound = bot.send_message(message.chat.id,
              'жду от тебя сообщение для озвучки\n\nесли ты уже писал его в запросе, повтори, пожалуйста, я не расслышал')

      username = message.chat.username
      if os.path.exists(f'{username}_output.ogg'):
        os.remove(f'{username}_output.ogg')  
      bot.register_next_step_handler(for_sound, voicing)

    elif modus == 3:
        '''ничего не будет, пока мы снова не напишем старт'''
        bot.send_message(message.chat.id, 'пока, увидимся')

def getting_time(text):
  text = preprocess(text) 
  for word in text.split(' '):
    for en, val in times_marks.items():
      if word in val:
        return en

def getting_timing(text):
  if getting_time(text): # сначала смотрим встречаются ли там названия типа сегодня завтра
    return getting_time(text)
  dates_extractor = DatesExtractor(morph_vocab) # если не встречаются экстрактим с помощью наташи
  times = []
  for inter in list(dates_extractor(text)):
    now = datetime.date.today()
    year = inter.fact.year or now.year
    month = inter.fact.month or now.month
    day = inter.fact.day or now.day
    times.append(datetime.date(year=year,month=month,day=day))
  return times 

def getting_cuts(text):
  text = preprocess(text) 
  for word in text.split(' '):
    for en, val in cuts.items():
      if word in val:
        return en

def getting_loc(text):
  if getting_cuts(text): # сначала смотрим встречаются ли там сокращения
    return getting_cuts(text)
  else: # если не встречаются экстрактим с помощью наташи
    for sent in sent_tokenize(text): 
      doc = (nlp(sent))
      for el in doc.sentences:
        for ent in el.entities:
          if (ent.type in ('LOC')): # ищем метку лок
            return morph.parse(ent.text)[0].normal_form.capitalize() # лемматизируем и повышаем регистр



def voicing(message):
    username = message.chat.username 
    v_req = bot.send_message(message.chat.id, 'запрос обрабатывается')
    tts = gTTS(text=message.text, lang='ru', slow=False)
    tts.save(f'{username}_output.ogg') # сохраняем для каждого пользователя, 
    #иначе на локальном сервере была бы беда у бота, что кому отправлять, когда с ботом работает несколько пользователей
    raw_audio = open(f'{username}_output.ogg', 'rb')

    bot.send_voice(message.chat.id, raw_audio)
    time.sleep(2) #ждём 
    request_again = bot.send_message(message.chat.id, 'хочешь чего-то ещё?')
    bot.register_next_step_handler(request_again, getting_text)
    return 
    
def is_russian(text):
    if re.search('/^[a-zA-Z][a-zA-Z\s-]*$/', text):
        return True
    else:
        return False

In [23]:
bot.infinity_polling()

2023-02-04 09:15:12,383 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-02-04 09:15:12,388 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
